# Feature extraction

In [3]:
addpath('../Classes','../Functions')
set(0, 'defaultFigureRenderer', 'painters')
warning('off','all')

### Set parameters

In [5]:
sorting_path = "../Data/Example_sorting";
params.QC.amp = nan;
params.QC.rate = [0.1 10];
params.QC.rv = 0.02;
params.Bursts.N = 0.0015;
params.Bursts.ISI_N = 1.5;
params.Bursts.merge_t = 2;
params.Regularity.binning = 0.1;
params.Bursts.binning = 0.1;

In [7]:
nw = WholeNetwork(sorting_path,params);

Unrecognized function or variable 'readNPY'.

Error in WholeNetwork/getCoordinates (line 155)
            xy = readNPY(fullfile(obj.InputPath, 'channel_positions.npy'));

Error in WholeNetwork (line 68)
                nw.XYElectrodes = nw.getCoordinates();

